In [1]:
%pip install dspy pydantic typing-extensions typing

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install -U mcp2py fastmcp

Note: you may need to restart the kernel to use updated packages.


In [37]:
from mcp2py import load

api = load('http://localhost:4000/mcp', headers={"Authorization": "Bearer sk-1234"})

scrape_tool = api.tools[0]

print(scrape_tool.__name__)




firecrawl_scrape


In [ ]:
import dspy
from pydantic import BaseModel, Field
from typing import Any, Literal, Optional, Type

class Role(BaseModel):
    name: str = Field(description="The name of the role")
    role: str = Field(description="The role of the role")
    allow_instructions: bool = Field(description="Whether the role allows instructions")
    instructions: str | None = Field(description="The instructions for the role")

class Expectation(BaseModel):
    """
    An input that a runnable can expect.
    """
    type: Literal["string", "number", "boolean", "array", "object"] = Field(description="The type of the expectation")
    name: str = Field(description="The name of the expectation")
    default: Optional[Any] = Field(description="The default value of the expectation")
    description: str = Field(description="A description of the expectation")

class Returnable(BaseModel):
    type: Literal["string", "number", "boolean", "array", "object"] = Field(description="The type of the returnable")
    name: str = Field(description="The name of the returnable")
    description: str = Field(description="A description of the returnable")


def expectation_to_field_string(expectation: Expectation, custom_class: Optional[Type] = None) -> str:
    """
    Convert an Expectation to a DSPy field definition string.
    """
    type_map = {
        "string": "str",
        "number": "float",
        "boolean": "bool",
        "array": "list",
        "object": custom_class.__name__ if custom_class else "dict"
    }
    python_type = type_map[expectation.type]
    field_def = f'{expectation.name}: {python_type} = dspy.InputField(desc="{expectation.description}")'
    return field_def

def returnable_to_field_string(returnable: Returnable, custom_class: Optional[Type] = None) -> str:
    """
    Convert a Returnable to a DSPy field definition string.
    """
    type_map = {
        "string": "str",
        "number": "float",
        "boolean": "bool",
        "array": "list",
        "object": custom_class.__name__ if custom_class else "dict"
    }
    python_type = type_map[returnable.type]
    # Use OutputField for return values
    field_def = f'{returnable.name}: {python_type} = dspy.OutputField(desc="{returnable.description}")'
    return field_def


class Runnable(BaseModel):
    expects: dict[str, Expectation] = Field(description="The expected inputs of the runnable")
    name: str = Field(description="The name of the runnable")
    returns: dict[str, Returnable] = Field(description="The expected outputs of the runnable")
    defined_role: bool = Field(description="Whether the runnable has a defined role")
    role: Role = Field(description="If defined_role is true, the role object")
    fly_role: str = Field(description="If defined_role is false, the string role of the runnable")

    def generate_role(self) -> str:
        """Generate the role string for this runnable."""
        has_defined_role = self.defined_role 
        runnable_role = self.role.role if has_defined_role else self.fly_role
        return runnable_role

    def generate_expectations(self) -> list[str]:
        """Generate field strings for all expectations."""
        return [expectation_to_field_string(exp) for exp in self.expects.values()]

    def generate_returns(self) -> list[str]:
        """Generate field strings for all returns."""
        return [returnable_to_field_string(ret) for ret in self.returns.values()]

    def generate_module(self) -> Type[dspy.Signature]:
        """
        Generate a DSPy Signature class based on expectations and returns.
        
        Returns:
            A dspy.Signature subclass
        """
        # Get the role for the signature docstring
        role = self.generate_role()
        
        # Build the signature class dynamically
        signature_name = f"{self.name}Signature"
        class_lines = [
            f"class {signature_name}(dspy.Signature):",
            f'    """{role}"""',
        ]
        
        # Add input fields
        for exp in self.expects.values():
            field_str = expectation_to_field_string(exp)
            class_lines.append(f"    {field_str}")
        
        # Add output fields
        for ret in self.returns.values():
            field_str = returnable_to_field_string(ret)
            class_lines.append(f"    {field_str}")
        
        # Create the signature class
        signature_code = "\n".join(class_lines)
        namespace = {"dspy": dspy}
        exec(signature_code, namespace)
        signature_class = namespace[signature_name]
        
        return signature_class


# Example usage:
if __name__ == "__main__":
    # Create a sample runnable
    runnable = Runnable(
        name="SearchAgent",
        expects={
            "query": Expectation(
                type="string",
                name="query",
                default=None,
                description="The search query to execute"
            ),
            "max_results": Expectation(
                type="number",
                name="max_results",
                default=10,
                description="Maximum number of results"
            )
        },
        returns={
            "results": Returnable(
                type="string",
                name="results",
                description="The search results"
            )
        },
        defined_role=False,
        role=Role(name="", role="", allow_instructions=False, instructions=None),
        fly_role="You are a helpful search agent that processes queries and returns relevant results."
    )
    
    # Generate the signature
    signature = runnable.generate_module()
    print(f"Generated signature: {signature}")
    print(f"Signature name: {signature.__name__}")
    
    # You can now use it with a predictor
    predictor = dspy.Predict(signature)
    # result = predictor(query="example", max_results=5)

Writing runnable.py


In [ ]:
from .runnable import Runnable

